In [1]:
%matplotlib inline
import random


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

from data_load import load_data
from utils import *

import gurobipy as gp
from gurobipy import GRB, tuplelist
import numpy as np

m = gp.Model(name="Re-scheduling")

Using license file C:\Users\Uyen\gurobi.lic
Academic license - for non-commercial use only - expires 2021-02-06


## A. Khởi tạo các tập dữ kiện

1. df_Tutor: Tập G các giảng viên tham gia giảng dạy tại trường
2. df_Course: Tập M các môn học được giảng dạy tại trường
3. df_Class: Tập L các lớp học 
4. df_Timetable: Tập B các buổi học trong học kỳ. Bao gồm thông tin: số thứ tự tuần, từ ngày... đến ngày..., thứ trong tuần, tiết bắt đầu 
5. df_Schedule: Thời khóa biểu hiện tại
6. old_schedule_for_class: Thời khóa biểu B_i tương ứng cho mỗi lớp l_i (chỉ lấy phần cần xếp lại tkb)
7. old_schedule_for_tutor: Thời khóa biểu B_k tương ứng cho mỗi giảng viên g_k (chỉ lấy phần cần xếp lại tkb)
8. max_session_to_add: số buổi cần học của module m_j cho lớp l_i

In [2]:
df_Schedule, df_Tutor, df_Class, df_Course, df_Timetable = load_data("C:\\Users\\Uyen\\Documents\\AdvancedAlg\\data\\Data_TK_bieu.xlsx")

In [3]:
# Counting

count_Schedule = df_Schedule.shape[0]
count_Tutor = df_Tutor.shape[0]
count_Class = df_Class.shape[0]
count_Course = df_Course.shape[0]
count_Session = df_Timetable.shape[0]

count_Schedule, count_Class, count_Tutor, count_Course, count_Session

(788, 131, 167, 2737, 280)

In [4]:
# Đọc dữ liệu input

input = pd.read_csv("C:\\Users\\Uyen\\Documents\\AdvancedAlg\\test_data\\1.csv", sep='\t')

In [5]:
i_classes = set(input.ID_Lop)
i_tutors = set(input.MaGV)

old_schedule_for_class = dict()
old_schedule_for_tutor = dict()


for cl in i_classes:
    filled_session = []
    df = df_Schedule[df_Schedule.ID_Lop==cl]
    for id, row in df.iterrows():
        sess = get_sessions_by_daterange(begin=row.start, end=row.end, timetable=df_Timetable, day=row.day, shift=row.beginning)
        filled_session += list(sess.values)
        
    old_schedule_for_class.update({cl: filled_session})


for t in i_tutors:
    filled_session = []
    df = df_Schedule[df_Schedule.MAGV==t]

    for id, row in df.iterrows():
        sess = get_sessions_by_daterange(begin=row.start, end=row.end, timetable=df_Timetable, day=row.day, shift=row.beginning)
        # print(sess)
        filled_session += list(sess.values)
    old_schedule_for_tutor.update({t: filled_session})

## B. Định nghĩa các biến quyết định

In [6]:
assign = tuplelist()

for i in range(input.shape[0]): 
    r_i = input.iloc[i]
    ass = [r_i.ID_Lop, r_i.MaMon, r_i.MaGV]
    sche = get_sessions_by_daterange(begin=r_i.start, end=r_i.end, timetable=df_Timetable)
    for j in range(len(sche)): 
        assign.append(tuple(ass + [sche[j]]))

assignment = m.addVars(assign, vtype = GRB.BINARY, name = "assignment")

In [7]:
assignment

.202', 194): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 195): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 196): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 197): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 198): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 199): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 200): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 201): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 202): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 203): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 204): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 205): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 206): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 207): <gurobi.Var *Awaiting Model Update*>,
 (3, '02.01.18', 'C.202', 208): <gurobi

## C. Định nghĩa hàm mục tiêu

In [7]:
obj = gp.quicksum(assignment[ass] for ass in assign)
m.setObjective(obj, GRB.MAXIMIZE)

## D. Khai báo các ràng buộc

In [8]:
# Constrain 1: Mỗi lớp chỉ học được một 1 môn tại mỗi thời điểm
# m.addConstrs( (assignment[(cl, c, t, s)] == 0 for (cl, c, t, s) in assign if s in old_schedule_for_class[cl]), name="constraint_2")


# # Constrain 2: Mỗi giáo viên chỉ dạy 1 môn tại mỗi thời điểm (có thể dạy nhiều lớp)
m.addConstrs( (assignment[(cl, c, t, s)] == 0 for (cl, c, t, s) in assign if s in old_schedule_for_tutor[t]), name="constraint_3")

# # Constrain 3: Đáp ứng đủ khối lượng học của các môn (Số buổi <= Số buổi cần học)
# max_session_to_add = dict( [ ((cl, c),n)  for cl, c, n in input[["ID_Lop", "MaMon", "SoBuoiBu"]].values] )
# m.addConstrs( (assignment.sum(cl, c, '*', '*') <= max_session_to_add[(cl, c)] for (cl, c) in max_session_to_add.keys()), name="constrain_4")

{(3, '00.00.18', 'C.705', 181): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 184): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 186): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 191): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 194): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 196): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 201): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 204): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 206): <gurobi.Constr *Awaiting Model Update*>,
 (3, '00.00.18', 'C.705', 216): <gurobi.Constr *Awaiting Model Update*>,
 (3, '01.20.34', 'C.904', 181): <gurobi.Constr *Awaiting Model Update*>,
 (3, '01.20.34', 'C.904', 184): <gurobi.Constr *Awaiting Model Update*>,
 (3, '01.20.34', 'C.904', 186): <gurobi.Constr *Awaiting Model Update*>,
 (3, '01.20.34', 'C.904', 191): <gurobi.Constr *Awa

In [17]:
m.update()
m.write('assignment_model.lp')

## E. Kết quả tối ưu mô hình

In [10]:
# Model optimization
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 115 rows, 700 columns and 808 nonzeros
Model fingerprint: 0x333bd8d0
Variable types: 0 continuous, 700 integer (700 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 1e+01]
Found heuristic solution: objective 50.0000000
Presolve removed 115 rows and 700 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 12 available processors)

Solution count 1: 50 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+01, best bound 5.000000000000e+01, gap 0.0000%


In [89]:
#Analysis
old_schedule = dict()

for id, row in df_Schedule.iterrows():
    sessions = get_sessions_by_daterange(begin=row.start, 
                end=row.end, 
                timetable=df_Timetable, 
                day=row.day, 
                shift=row.beginning).values
    tutor = row.MAGV
    clas = row.ID_Lop
    course = row.MaMon

    for s in sessions:
        try:
            old_schedule[s].append((clas, course, tutor))
        except: 
            old_schedule.update( {s: [(clas, course, tutor)]} )
    

In [28]:
for v in m.getVars()[0:10]:
    # print(v.varName.)
    v.VarName.
    print('%s %g' % (v.varName, v.x))

assignment[3,00.00.18,C.705,180] 0
assignment[3,00.00.18,C.705,181] 0
assignment[3,00.00.18,C.705,182] 0
assignment[3,00.00.18,C.705,183] 0
assignment[3,00.00.18,C.705,184] 0
assignment[3,00.00.18,C.705,185] 0
assignment[3,00.00.18,C.705,186] 0
assignment[3,00.00.18,C.705,187] 0
assignment[3,00.00.18,C.705,188] 0
assignment[3,00.00.18,C.705,189] 0


In [27]:
assignment

273] (value 1.0)>,
 (3,
  '02.01.18',
  'C.202',
  274): <gurobi.Var assignment[3,02.01.18,C.202,274] (value 1.0)>,
 (3,
  '02.01.18',
  'C.202',
  275): <gurobi.Var assignment[3,02.01.18,C.202,275] (value 1.0)>,
 (3,
  '02.01.18',
  'C.202',
  276): <gurobi.Var assignment[3,02.01.18,C.202,276] (value 1.0)>,
 (3,
  '02.01.18',
  'C.202',
  277): <gurobi.Var assignment[3,02.01.18,C.202,277] (value 1.0)>,
 (3,
  '02.01.18',
  'C.202',
  278): <gurobi.Var assignment[3,02.01.18,C.202,278] (value 1.0)>,
 (3,
  '02.01.18',
  'C.202',
  279): <gurobi.Var assignment[3,02.01.18,C.202,279] (value 1.0)>,
 (3,
  '02.01.64',
  'C.202',
  180): <gurobi.Var assignment[3,02.01.64,C.202,180] (value 0.0)>,
 (3,
  '02.01.64',
  'C.202',
  181): <gurobi.Var assignment[3,02.01.64,C.202,181] (value 0.0)>,
 (3,
  '02.01.64',
  'C.202',
  182): <gurobi.Var assignment[3,02.01.64,C.202,182] (value 0.0)>,
 (3,
  '02.01.64',
  'C.202',
  183): <gurobi.Var assignment[3,02.01.64,C.202,183] (value 0.0)>,
 (3,
  '02.

In [100]:
for i, j, k, t in assign:
    if assignment[(i, j, k, t)]==0: print(cb)

GurobiError: Constraint has no bool value (are you trying "lb <= expr <= ub"?)

In [19]:
# status = model.status
# if status == GRB.UNBOUNDED:
#     print('The model cannot be solved because it is unbounded')
#     sys.exit(0)
# if status == GRB.OPTIMAL:
#     print('The optimal objective is %g' % model.objVal)
#     sys.exit(0)
# if status != GRB.INF_OR_UNBD and status != GRB.INFEASIBLE:
#     print('Optimization was stopped with status %d' % status)
#     sys.exit(0)

### Stage 2: Có xét đến độ ưu tiên của các môn khi sắp lại lịch